## Aprendizado sobre tools de OpenAI

### Le o arquivo com o conteudo do arquivo

In [222]:
import openai
import json
import os
from dotenv import find_dotenv, load_dotenv

# load the api key
_ = load_dotenv(find_dotenv())
client = openai.OpenAI(api_key=os.getenv("API_KEY"))
list_of_dicts = []

def parse_line(line):
    result = {}
    # Split with | delimiter
    parts = line.strip().split('|')
    
    if len(parts) >= 2:
        result['email'] = parts[0].strip()
        # Check if pwd has space for comments
        pwd_parts = parts[1].strip().split(' ', 1)
        result['pwd'] = pwd_parts[0]
        
        # Add comment if exists
        if len(pwd_parts) > 1:
            result['comment'] = pwd_parts[1]
    
    return result

def list_of_email_password(file_path="pwd.txt"):
    list_of_dicts.clear()
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Removes the new line character if necessary
                line = line.strip()
                #Process the line
                parsed = parse_line(line)
                list_of_dicts.append(parsed)
                # print(parsed)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return list_of_dicts
        

Usando a função

In [223]:
list_of_email_password('pwd.txt')

[{'email': 'djskaner@hotmail.com', 'pwd': 'rockgpzt'},
 {'email': 'aline.limp@gmail.com', 'pwd': '151213aej'},
 {'email': 'wilsongilberto@terra.com.br', 'pwd': '1959ywlk'},
 {'email': 'wilsongilberto@terra.com.br', 'pwd': '1959ywlk'},
 {'email': 'Boleti_bep@hotmail.com', 'pwd': '524718ml'},
 {'email': 'agiginola@bol.com.br', 'pwd': 'isabela9'},
 {'email': 'sonia.osorio@globo.com', 'pwd': 'c8c2i2e1'},
 {'email': 'klebergs789@gmail.com', 'pwd': 'matheuskauan'},
 {'email': 'gerlanemsl@hotmail.com', 'pwd': 'edenilson'},
 {'email': 'thiagomarques@hotmail.co.uk', 'pwd': 'technicolorvh1!'},
 {'email': 'belmugo@gmail.com', 'pwd': '03011404'}]

Para o modelo devemos transformar o dicionário em string com `json.dumps()`

In [224]:
json.dumps(list_of_dicts)

'[{"email": "djskaner@hotmail.com", "pwd": "rockgpzt"}, {"email": "aline.limp@gmail.com", "pwd": "151213aej"}, {"email": "wilsongilberto@terra.com.br", "pwd": "1959ywlk"}, {"email": "wilsongilberto@terra.com.br", "pwd": "1959ywlk"}, {"email": "Boleti_bep@hotmail.com", "pwd": "524718ml"}, {"email": "agiginola@bol.com.br", "pwd": "isabela9"}, {"email": "sonia.osorio@globo.com", "pwd": "c8c2i2e1"}, {"email": "klebergs789@gmail.com", "pwd": "matheuskauan"}, {"email": "gerlanemsl@hotmail.com", "pwd": "edenilson"}, {"email": "thiagomarques@hotmail.co.uk", "pwd": "technicolorvh1!"}, {"email": "belmugo@gmail.com", "pwd": "03011404"}]'

Criando as tools para o modelo

In [225]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_of_email_password",
            "description": "Get the email list and password",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The path to the file"
                    }
                },
                "required": ["file_path"]
            }
        }
    }
]

avaliable_tools = {
    "list_of_email_password": list_of_email_password
}



Chama o modelo de linguagem do OpenAI

In [253]:
messages = [
    {"role": "user", "content": "Quais os domínios dos emails?"}
]

file_path = "pwd.txt"
result = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    )

In [254]:
result

ChatCompletion(id='chatcmpl-B77H7XXYg9lieM7M9wJBJcdTo0sN6', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ra14v5C4EAbpj2qkcUyTWz9Q', function=Function(arguments='{"file_path":"data/emails.txt"}', name='list_of_email_password'), type='function')]))], created=1741035961, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=65, total_tokens=86, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Valida se vai precisar usar alguma função fornecida

In [255]:
message_result = result.choices[0].message
tool_calls = message_result.tool_calls


if tool_calls:
    messages.append(message_result)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = avaliable_tools[function_name]
        # function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(file_path=file_path)
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": json.dumps(function_response)
        })
        print(function_response)

[{'email': 'djskaner@hotmail.com', 'pwd': 'rockgpzt'}, {'email': 'aline.limp@gmail.com', 'pwd': '151213aej'}, {'email': 'wilsongilberto@terra.com.br', 'pwd': '1959ywlk'}, {'email': 'wilsongilberto@terra.com.br', 'pwd': '1959ywlk'}, {'email': 'Boleti_bep@hotmail.com', 'pwd': '524718ml'}, {'email': 'agiginola@bol.com.br', 'pwd': 'isabela9'}, {'email': 'sonia.osorio@globo.com', 'pwd': 'c8c2i2e1'}, {'email': 'klebergs789@gmail.com', 'pwd': 'matheuskauan'}, {'email': 'gerlanemsl@hotmail.com', 'pwd': 'edenilson'}, {'email': 'thiagomarques@hotmail.co.uk', 'pwd': 'technicolorvh1!'}, {'email': 'belmugo@gmail.com', 'pwd': '03011404'}]


In [256]:
messages

[{'role': 'user', 'content': 'Quais os domínios dos emails?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ra14v5C4EAbpj2qkcUyTWz9Q', function=Function(arguments='{"file_path":"data/emails.txt"}', name='list_of_email_password'), type='function')]),
 {'tool_call_id': 'call_ra14v5C4EAbpj2qkcUyTWz9Q',
  'role': 'tool',
  'name': 'list_of_email_password',
  'content': '[{"email": "djskaner@hotmail.com", "pwd": "rockgpzt"}, {"email": "aline.limp@gmail.com", "pwd": "151213aej"}, {"email": "wilsongilberto@terra.com.br", "pwd": "1959ywlk"}, {"email": "wilsongilberto@terra.com.br", "pwd": "1959ywlk"}, {"email": "Boleti_bep@hotmail.com", "pwd": "524718ml"}, {"email": "agiginola@bol.com.br", "pwd": "isabela9"}, {"email": "sonia.osorio@globo.com", "pwd": "c8c2i2e1"}, {"email": "klebergs789@gmail.com", "pwd": "matheuskauan"}, {"email": "gerlanemsl@hotmail.com", "pwd": "edenilson"}, {"email": 

Obtém a segunda resposta do modelo, agora sem a necessidade de passar a tools

In [257]:
final_output = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages
)
print(final_output)

ChatCompletion(id='chatcmpl-B77H8PGSG9k5sNkmC7kSRdjlw0WN7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Os domínios dos emails presentes no arquivo são: hotmail.com, gmail.com, terra.com.br, bol.com.br e globo.com.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1741035962, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=32, prompt_tokens=285, total_tokens=317, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [258]:
output = final_output.choices[0].message
print(output.content)

Os domínios dos emails presentes no arquivo são: hotmail.com, gmail.com, terra.com.br, bol.com.br e globo.com.
